In [1]:
import requests
import pandas as pd
import lxml.etree as ET
from bs4 import BeautifulSoup as soup
import unicodedata

In [ ]:
df_all = pd.read_hdf("pandemie_titeldaten.h5", key="df")
df_all

### Optional: Einschränken des Sets auf deutschsprachige Publikationen: 

In [ ]:
df = df_all.loc[df_all.lang == "ger"]
df

### Optional: Kleinschreibung

In [ ]:
df.loc[:, 'title'] = df['title'].str.lower() 

## Entfernung von "Stopwords": 

In [ ]:
#Importieren der zu entfernenden Stopwörter (wie bpsw. "und", "der", etc.):

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
#nltk.download('stopwords')

stopger = stopwords.words('german')

#Bei Bedarf können weitere Stopwords hinzugefügt werden: 
newStopwords = ['']
stopger.extend(newStopwords)

In [ ]:
#Entfernung überflüssiger Leerzeichen sowie Kommata, Doppelpunkte und eckige Klammern:
df.loc[:, 'title_clean'] = df['title'].str.replace(r'\s+', ' ', regex=True)
df.loc[:, 'title_clean'] = df['title_clean'].str.replace(r'(\w+)[,.:?!\[\]]', r'\1', regex=True)
df.loc[:, 'title_clean'] = df['title_clean'].str.replace(" - ", "") 
df

In [ ]:
#Abgleich der Wörter mit den Stopwörtern - Stopwerds werden ausgeschlossen: 
df.loc[:, 'title_clean'] = df['title_clean'].apply(lambda x: ' '.join([word for word in x.split() if word not in stopger]))
df

## Näherer Eindruck der Titel - ggf. weitere Bereinigung: 

In [ ]:
df.title_clean

In [ ]:
df = df.drop_duplicates(subset='title', keep='first')

In [ ]:
df.title_clean

In [ ]:
for title in df.title_clean:
    print(title)

## Ein erster Eindruck (Verteilung nach Jahreszahlen):


In [ ]:
#Mit der Bibliothek plotly können interaktive Graphen geplottet werden:
import plotly.express as px

fig1 = px.histogram(df, x='date') #Mit px.bar() wird jede Zeile des übergebenen DataFrames in Form eines Balken dargestellt. Die Notation wird wie folgt übergeben: dataframe, x-Achse, y-Achse.
fig1.show() #Mit der Funktion .show() wird der fertige Graph angezeigt.


In [ ]:
df.query("date.str.len() > 4")

OK, more datacleaning first... :

In [ ]:
df['date'] = df['date'].str.replace(r'[\[\]\(\)?]', '', regex=True)

In [ ]:
df.query("date.str.len() > 4")

In [ ]:
fig2 = px.histogram(df, x="date", barmode='group')
fig2.update_layout(barmode='stack', xaxis={'categoryorder':'category ascending'})
fig2.show()

In [ ]:
df.loc[:, 'date'] = df['date'].str.replace(r'-+$', '', regex=True)

In [ ]:
df.loc[:, 'date'] = df['date'].str[-4:]

In [ ]:
#df.loc[df["DATE"] == "20ka"]

In [ ]:
df.loc[:, 'date'] = df['date'].str.replace("20ka", "2020")

In [ ]:
fig3 = px.histogram(df, x="date", barmode='group')
fig3.update_layout(barmode='stack', xaxis={'categoryorder':'category ascending'})
fig3.show()

In [ ]:
df

Zwischenspeichern des Ergebnisses: 

In [ ]:
df.to_hdf("df_prep.h5", key="df")